In [6]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import os
mnist = input_data.read_data_sets("/home/zhaocq/桌面/tensorflow/mnist/raw/",one_hot=True)
sess = tf.InteractiveSession()
trX, trY, teX, teY = mnist.train.images, mnist.train.labels,mnist.test.images,mnist.test.labels
X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,10])
#定义权重初始化函数以及权重向量
#def init_weights(shapes):
#    return tf.Variable(tf.random_normal(shapes, stddev=0.01))
#w_h = init_weights([784,625])
#w_h2 = init_weights([625,625])
#w_o = init_weights([625,10])
w_h = tf.Variable(tf.random_normal([784,625], stddev=0.01),name = 'w_1')
w_h2 = tf.Variable(tf.random_normal([625,625], stddev=0.01),name = 'w_2')
w_o = tf.Variable(tf.random_normal([625,10], stddev=0.01),name = 'w_0')
variables_dict = {'w_1':w_h, 'w_2':w_h2,  'w_o':w_o}

Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-images-idx3-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-labels-idx1-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-labels-idx1-ubyte.gz


In [7]:
#定义模型
def model(X, w_h , w_h2 , w_o, p_keep_input,p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))
    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h,w_h2))
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    return tf.nn.softmax(tf.matmul(h2 ,w_o))
p_keep_input = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)
py_x = model(X, w_h,w_h2,w_o,p_keep_input,p_keep_hidden)
#损失函数定义
cost = tf.reduce_mean(-tf.reduce_mean(Y*tf.log(py_x),reduction_indices=[1]))
train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
predict_op = tf.equal(tf.argmax(py_x,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(predict_op,tf.float32))

In [8]:
 ckpt_dir = "./trainsave/ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
global_step = tf.Variable(0,name='global_step',trainable = False)
saver = tf.train.Saver(variables_dict)
non_storable_variable = tf.Variable(777)


In [9]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    start = global_step.eval()
    for i in range(start, 1000):
        batch = mnist.train.next_batch(100)
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={X: batch[0],Y:batch[1],p_keep_hidden:0.8,p_keep_input:0.5}) #keep_prob训练时通常小于1，测试时为1
            print("step %d,training accuracy %g"%(i,train_accuracy))
        sess.run(train_op,feed_dict={X: batch[0],Y:batch[1],p_keep_hidden:0.8,p_keep_input:0.5})
        #for start, end in zip(range(0,len(trX),128),range(128,len(trX)+1),128):
            #sess.run(train_op,feed_dict={X:trX[start:end],Y:trY[start:end],p_keep_hidden:0.8,p_keep_input:0.5})
        global_step.assign(i).eval()
        saver.save(sess, ckpt_dir + "/model.ckpt",global_step=global_step)
    trainaccuracy = sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels,p_keep_hidden:1.0,p_keep_input:1.0})
    print("test accuracy %g"%trainaccuracy)#feed_dict给placeholder创建的tensor赋值

step 0,training accuracy 0.08
step 100,training accuracy 0.11
step 200,training accuracy 0.76
step 300,training accuracy 0.78
step 400,training accuracy 0.96
step 500,training accuracy 0.93
step 600,training accuracy 0.94
step 700,training accuracy 0.95
step 800,training accuracy 0.94
step 900,training accuracy 0.9
test accuracy 0.9589
